# PYPROTOLINC Demo

This notebook demonstrates how the different concepts within `pyprotolinc` can be used explaining various
customization options.

In [81]:
# we will need a bunch of imports for the below
from enum import unique

import numpy as np

from pyprotolinc.models.state_models import show_state_models, AbstractStateModel, state_model_by_name
import pyprotolinc._actuarial as actuarial
from pyprotolinc.riskfactors.risk_factors import get_risk_factor_names, Gender, SmokerStatus
from pyprotolinc.assumptions.iohelpers import AssumptionsLoaderFromConfig

## 1. State Models
As a first step a states model needs to be selected. We can choose between the built in states models or create a new one.

In [2]:
state_models = show_state_models()
state_models

{'AnnuityRunoffStates': <enum 'AnnuityRunoffStates'>,
 'MortalityStates': <enum 'MortalityStates'>,
 'MultiStateDisabilityStates': <enum 'MultiStateDisabilityStates'>}

We can look at the description of each of the built in models:

In [3]:
for state_model_name, state_model_class in state_models.items():
    #print(state_model_name)
    print(state_model_class.describe())
    print()

AnnuityRunoffStates:  A state model consisting of two states:
        - DIS1 (=0) representing the annuity phase
        - DEATH (=1)
    

MortalityStates:  A state model with four states that can be used to model simple mortality term/perm products.
        - ACTIVE = 0
        - DEATH = 1
        - LAPSED = 2
        - MATURED = 3
    

MultiStateDisabilityStates:  A state model for a disabiility product with two disabled states. 
        - ACTIVE = 0
        - DIS1 = 1
        - DIS2 = 2
        - DEATH = 3
        - LAPSED = 4
    



We can now decide for one of the built-in models or create a custom model.

In [7]:
# decide for an existing one
# selected_state_model = state_model_by_name('MortalityStates')

# or create a new one
@unique
class DeferredAnnuityStates(AbstractStateModel):
    """ A state model for a deferred annuities. 
        - ACTIVE = 0
        - DEAD = 1
        - LAPSED = 2
        - ANNUITANT = 3
    """
    ACTIVE = 0
    DEAD = 1
    LAPSED = 2
    ANNUITANT = 3  # 6

# the new state model is now registered in pyprotolinc
show_state_models()

{'AnnuityRunoffStates': <enum 'AnnuityRunoffStates'>,
 'MortalityStates': <enum 'MortalityStates'>,
 'MultiStateDisabilityStates': <enum 'MultiStateDisabilityStates'>,
 'DeferredAnnuityStates': <enum 'DeferredAnnuityStates'>}

In [9]:
selected_state_model = state_model_by_name('DeferredAnnuityStates')

# print out the states
for state in selected_state_model:
    print(state)

DeferredAnnuityStates.ACTIVE
DeferredAnnuityStates.DEAD
DeferredAnnuityStates.LAPSED
DeferredAnnuityStates.ANNUITANT


### Mapping the States to the Standard Output Model
The next thing we need to specify is how the state should be mapped to the standatd output model
(if there is need for that at all)

In [ ]:
# todo

## 2. Supplying Valuation Assumptions

We need to provide assumptions for the various state transitions in the model. In `pyprotolinc` this
is done by creating an object of type `AssumptionSet`. AssupmtionSets ar composed of `AssumptionProvides` and the the selection depends on `RiskFactors`.

### Risk Factors

Currently `pyprotolinc`supports five risk factors.

In [61]:
from pyprotolinc.riskfactors.risk_factors import _C_RISK_FACTORS
_C_RISK_FACTORS

{'Age': <CRiskFactors.Age: 0>,
 'Gender': <CRiskFactors.Gender: 1>,
 'CalendarYear': <CRiskFactors.CalendarYear: 2>,
 'SmokerStatus': <CRiskFactors.SmokerStatus: 3>,
 'YearsDisabledIfDisabledAtStart': <CRiskFactors.YearsDisabledIfDisabledAtStart: 4>}

In [28]:
# to better understand how the risk factors are encoded we can look at the values, e.g
[(g.name, g.value) for g in Gender]

[('M', 0), ('F', 1)]

In [29]:
[(g.name, g.value) for g in SmokerStatus]

[('S', 0), ('N', 1), ('A', 2), ('U', 3)]

### Building an Assumption Set

In [64]:
# the assumption set must have the dimension of the state models
states_dimension = len(selected_state_model)
acs = actuarial.AssumptionSet(states_dimension);

In [65]:
# to query the assumption set we need to provide values for each risk factor

# create a dummy realization of the risk factors
risk_factors = np.zeros(5, dtype=np.int32)
risk_factors[0] = 32              # age
risk_factors[1] = Gender.F        # gender
risk_factors[2] = 2023            # CalendarYear
risk_factors[3] = SmokerStatus.N  # SmokerStatus
risk_factors[4] = 2023            # YearsDisabledIfDisabledAtStart

risk_factors

array([  32,    1, 2023,    1, 2023])

In [66]:
# at this stage we can query
acs.get_single_rateset(risk_factors).reshape((states_dimension, states_dimension))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

All possible state transitions have a probability of zero.

### Providers

Assumption Providers are multidimensional lookup tables that specify the transition probabilities for a single state transition depending on the risk factors.

In [49]:
# we programmaticall specify a provider that depends on Gender and Age

vals2D = np.array([
       [0.1, 0.2, 0.3, 0.4],  # specifies Gender M and ages 0, 1, 2, 3
       [1.1, 1.2, 1.3, 1.4]   # specifies Gender M and ages 0, 1, 2, 3
], dtype=np.float64)


offsets = np.zeros(2, dtype=np.int32)
provider_test = actuarial.StandardRateProvider([actuarial.CRiskFactors.Gender, actuarial.CRiskFactors.Age], vals2D, offsets)

In [50]:
# we can query the provider for gender F and age 2
provider_test.get_rate([Gender.F, 2])

1.3

In [74]:
# we create a dummy provider programmatically for all ages
vals2d_long = (np.arange(120 * 2)).reshape((2, 120)) / 1000.0
vals2d_long

array([[0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
        0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
        0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
        0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
        0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
        0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
        0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
        0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
        0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
        0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
        0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
        0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
        0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
        0.117, 0.118, 0.119],
       [0.12 , 0.121, 0.122, 0.123, 0.124, 0.12

In [75]:
provider = actuarial.StandardRateProvider([actuarial.CRiskFactors.Gender, actuarial.CRiskFactors.Age], vals2d_long, offsets)
provider.get_rate([Gender.F, 9])

0.129

In [76]:
# now we can add the provider to the assumption set for the state-transition 1 -> 0
acs.add_provider_std(1, 0, provider)

Now that we have added a provider we can query the assumption set again:

In [77]:
acs.get_single_rateset(risk_factors).reshape((states_dimension, states_dimension))

array([[0.   , 0.   , 0.   , 0.   ],
       [0.152, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ]])

The number we see is in the second row (corresponding with STATE#1) and gives us the
probability to transition to STATE#0 (first column).

### Building AssumptionSets from Files

Instead manually bootstrapping the required object there is a more convenient way for simpler cases to create 
assumptions sets from configruration files and spreadsheets.

In [83]:
assumption_config_loader = AssumptionsLoaderFromConfig(r"..\di_assumptions.yml")
# ass_config_loader.load(mb)

In [84]:
assumption_config_loader.assumptions_spec

{'be': [[0,
   3,
   ['FileTable', 'tests/base_assumption.xlsx', 'MORTALITY (0->3)']],
  [0, 4, ['FileTable', 'tests/base_assumption.xlsx', 'LAPSE (0->4)']],
  [0, 1, ['FileTable', 'tests/base_assumption.xlsx', 'DIS1 (0->1)']],
  [1, 2, ['FileTable', 'tests/base_assumption.xlsx', 'DIS_WORSEN (1->2)']],
  [1, 3, ['FileTable', 'tests/base_assumption.xlsx', 'DIS_DEATH1 (1->3)']],
  [1, 0, ['FileTable', 'tests/base_assumption.xlsx', 'REC1(1->0)']],
  [0, 2, ['FileTable', 'tests/base_assumption.xlsx', 'DIS2(0->2)']],
  [2, 1, ['FileTable', 'tests/base_assumption.xlsx', 'DIS_IMPR (2->1)']],
  [2, 3, ['FileTable', 'tests/base_assumption.xlsx', 'DIS_DEATH2 (2->3)']],
  [2, 0, ['FileTable', 'tests/base_assumption.xlsx', 'REC2(2->0)']]],
 'res': [[0,
   3,
   ['FileTable', 'tests/base_assumption.xlsx', 'MORTALITY (0->3)']],
  [0, 4, ['FileTable', 'tests/base_assumption.xlsx', 'LAPSE (0->4)']],
  [0, 1, ['FileTable', 'tests/base_assumption.xlsx', 'DIS1 (0->1)']],
  [1, 2, ['FileTable', 'tests/bas